In [ ]:
!python -m spacy download ru_core_news_md

In [21]:
from pathlib import Path
import pandas as pd

# pip install git+https://github.com/boudinfl/pke.git
# pke relies on spacy (>= 3.2.3) for text processing and requires models to be installed:
# # download the russian model
# python -m spacy download ru_core_news_md
import pke
import string
import ru_core_news_md
import spacy

In [22]:
spacy_model = spacy.load("ru_core_news_md")

In [27]:
# all df
dataframe = pd.read_csv("../../data/datasets/all_dataset.csv", sep=";", encoding="utf-8")
    
# otzyvus
dataframe = dataframe[dataframe["rubrics"].str.contains('образование_отзывус')].dropna().astype("str").reset_index()

# Yandex
# dataframe = dataframe[dataframe["rubrics"].str.contains('Гостиница')].dropna().astype("str").reset_index()

In [28]:
dataframe

,index,Unnamed: 0,text,rubrics
0,0,0,"Был скучноват. Много практики, решение задач п...",образование_онлайн_курсы;образование_отзывус
1,1,1,Первые пол семестра - упражнения всякие сценич...,образование_онлайн_курсы;образование_отзывус
2,2,2,"Чисто основы основ 3д моделинга, для тех кто в...",образование_онлайн_курсы;образование_отзывус
3,3,3,"Электив просто песня, две презентации за весь ...",образование_онлайн_курсы;образование_отзывус
4,4,4,Прекрасный электив помогающий разобраться в се...,образование_онлайн_курсы;образование_отзывус
...,...,...,...,...
1383,1383,1383,"Безумно интересный электив, один из самых мной...",образование_онлайн_курсы;образование_отзывус
1384,1384,1384,"Электив подходит экологам, химикам, биологам и...",образование_онлайн_курсы;образование_отзывус
1385,1385,1385,"Баллы ставятся за присутствие, закрыть легко, ...",образование_онлайн_курсы;образование_отзывус
1386,1386,1386,"Интересный предмет, позволяющий изучить механи...",образование_онлайн_курсы;образование_отзывус


In [25]:
text = []
for i in range(len(dataframe)):
    text.append([
        (w.text, w.pos_)
        for w in spacy_model(dataframe['text'][i])
    ])

text_normalized = []
for i in range(len(dataframe)):
    text_normalized.append([
        (w.lemma_, w.pos_)
        for w in spacy_model(dataframe['text'][i])
        if w.pos_ != "PUNCT"
    ])

In [26]:
print(text[0], '\n')
print(text_normalized[0])

[('Был', 'AUX'), ('скучноват', 'NOUN'), ('.', 'PUNCT'), ('Много', 'ADV'), ('практики', 'NOUN'), (',', 'PUNCT'), ('решение', 'NOUN'), ('задач', 'NOUN'), ('про', 'ADP'), ('заработную', 'ADJ'), ('плату', 'NOUN'), ('.', 'PUNCT'), ('Довольно', 'ADV'), ('сложно', 'ADJ'), ('зарабатывать', 'VERB'), ('баллы', 'NOUN'), (',', 'PUNCT'), ('нужно', 'ADJ'), ('теорию', 'NOUN'), ('учить', 'VERB'), ('и', 'CCONJ'), ('местами', 'NOUN'), ('её', 'PRON'), ('рассказывать', 'VERB'), (',', 'PUNCT'), ('решать', 'VERB'), ('постоянно', 'ADV'), ('задачи', 'NOUN'), (',', 'PUNCT'), ('выходить', 'VERB'), ('к', 'ADP'), ('доске', 'NOUN'), ('.', 'PUNCT'), ('Когда', 'SCONJ'), ('баллы', 'NOUN'), ('набраны', 'VERB'), ('на', 'ADP'), ('зачёт', 'NOUN'), (',', 'PUNCT'), ('все', 'ADV'), ('равно', 'ADJ'), ('нужно', 'ADJ'), ('ходить', 'VERB'), (',', 'PUNCT'), ('иначе', 'CCONJ'), ('она', 'PRON'), ('их', 'PRON'), ('уберёт', 'VERB'), ('(', 'PUNCT'), ('хотя', 'SCONJ'), (',', 'PUNCT'), ('она', 'PRON'), ('просто', 'PART'), ('так', 'ADV'

# TextRank

In [11]:
%%time

# define the set of valid Part-of-Speeches
# pos = {'NOUN', 'PROPN', 'ADJ'}
pos = {'NOUN', 'PROPN'}

# 1. create a TextRank extractor.
extractor_textrank = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor_textrank.load_document(input=text,
                        language='ru',
                        #spacy_model=spacy_model,
                        normalization=None)

extractor_textrank.ngram_selection(n=1)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor_textrank.candidate_weighting(window=3,
                              pos=pos,
                              top_percent=0.33)


CPU times: user 2.56 s, sys: 16 ms, total: 2.57 s
Wall time: 2.58 s


In [12]:
# 4. get the 10-highest scored candidates as keyphrases
keyphrases_textrank = extractor_textrank.get_n_best(n=100)
keyphrases_textrank

[('онлайн - прокторинг -', 0.037427318434282444),
 ('- студентов электив', 0.03164656345596668),
 ('теории - электив', 0.03067535107332989),
 ('онлайн - электив', 0.030369587955607565),
 ('электив - просто', 0.029741895981227416),
 ('- электив', 0.02923496434974802),
 ('электив -', 0.02907045434974802),
 ('преподаватель - практик', 0.028786566737951512),
 ('- преподаватель', 0.02856187724014948),
 ('преподаватель - очень', 0.02839222634044728),
 ('проблема - преподаватель', 0.028058003735029178),
 ('преподаватель -', 0.027808247240149478),
 ('все задания - зачёт', 0.024285323384568555),
 ('конце курса тест - прокторинг', 0.02414677122764213),
 ('задания - зачет', 0.02311096494907906),
 ('вопросов - доп баллы', 0.022790264501086836),
 ('- задания', 0.022362702224998025),
 ('минус - баллы', 0.022309007232912237),
 ('конце семестра -', 0.021982665277991923),
 ('задания -', 0.021888442224998025),
 ('- материала лекций', 0.021881612343788957),
 ('точки зрения gamedev - разработчика', 0.0217

# TopicRank

In [13]:
%%time

# initialize keyphrase extraction model, here TopicRank
extractor_topicrank = pke.unsupervised.TopicRank()

# 2. load the content of the document.
stoplist = list(string.punctuation)
stoplist += pke.lang.stopwords.get('ru')

# load the content of the document, here document is expected to be a simple
# test string and preprocessing is carried out using spacy
extractor_topicrank.load_document(input=text, language='ru', stoplist=stoplist)

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
pos = {'NOUN', 'PROPN'}
extractor_topicrank.candidate_selection(pos=pos)

# candidate weighting, in the case of TopicRank: using a random walk algorithm
extractor_topicrank.candidate_weighting(threshold=0.74, method='average')


CPU times: user 4min 8s, sys: 734 ms, total: 4min 9s
Wall time: 4min 9s


In [14]:
# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
keyphrases_topicrank = extractor_topicrank.get_n_best(n=100)
keyphrases_topicrank

[('электив', 0.04427377197566012),
 ('преподаватель', 0.029477457161863473),
 ('пары', 0.026975169703419614),
 ('задания', 0.01896115255592954),
 ('лекции', 0.014421865962461268),
 ('баллы', 0.013012493079078487),
 ('работы', 0.01214821555572097),
 ('тему', 0.01211651655891517),
 ('элективом', 0.011685028363897199),
 ('занятия', 0.010682888606754627),
 ('практики', 0.009877830234980725),
 ('курса', 0.009323988018060371),
 ('студентам', 0.008782146789407086),
 ('информации', 0.008541244813634339),
 ('презентации', 0.00853153503626849),
 ('зачёт', 0.008112553222784515),
 ('тест', 0.0077045806770978005),
 ('вопросы', 0.007544823720584581),
 ('материал', 0.0073176373933622136),
 ('уровень знаний', 0.007183868169714946),
 ('конце', 0.006371111831704594),
 ('целом', 0.0062637943495100195),
 ('предмета', 0.005565472431785979),
 ('языке', 0.0050795609741564425),
 ('проект', 0.0049513210189695445),
 ('группы', 0.004898641813444832),
 ('время', 0.004546976985854948),
 ('фильмы', 0.00452712425721

# SingleRank

In [15]:
%%time

# 1. create a SingleRank extractor.
extractor_singlerank = pke.unsupervised.SingleRank()

# 2. load the content of the document.
extractor_singlerank.load_document(input=text,
                        language='ru',
                        normalization=None)

# 3. select the longest sequences of nouns and adjectives as candidates.
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN'}
extractor_singlerank.candidate_selection(pos=pos)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk. In the graph, nodes are words of
#    certain part-of-speech (nouns and adjectives) that are connected if
#    they occur in a window of 10 words.
extractor_singlerank.candidate_weighting(window=3,
                                         pos=pos)


CPU times: user 341 ms, sys: 16 ms, total: 357 ms
Wall time: 355 ms


In [16]:
# 5. get the 100-highest scored candidates as keyphrases
keyphrases_singlerank = extractor_singlerank.get_n_best(n=100)
keyphrases_singlerank

[('онлайн - прокторинг - сдача экзамена', 0.04600195490786861),
 ('бла - бла - бла', 0.0450615888623166),
 ('у - жас - но', 0.04480167534563801),
 ('- студентов электив', 0.03822314775587534),
 ('онлайн - электив', 0.037158992295294214),
 ('теории - электив', 0.03714937161883851),
 ('электив - сплошь социология', 0.03642101111501326),
 ('лесу - электив', 0.03640393975871578),
 ('электив - чудо', 0.03636935177079664),
 ('электив - про', 0.03588315975871579),
 ('электив -', 0.035763520514121024),
 ('- электив', 0.035750200514121026),
 ('электив - пытка', 0.03569591975871579),
 ('повова ольга андреевна - милейший преподаватель', 0.03506313270108074),
 ('преподаватель - практик', 0.03446685381378412),
 ('- преподаватель', 0.03427714732802325),
 ('преподаватель - психолог', 0.034257270818163835),
 ('преподаватель - замечательная', 0.034096363494809635),
 ('преподаватель петров в.в. -', 0.034052922301383896),
 ('преподаватель - восхитительная', 0.03383188349480964),
 ('преподаватель - крутая

# PositionRank

In [17]:
%%time

# define the valid Part-of-Speeches to occur in the graph
pos = {'NOUN', 'PROPN'}

# define the grammar for selecting the keyphrase candidates
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"
# grammar = "NP: {<NOUN|PROPN>+}"

# 1. create a PositionRank extractor.
extractor_positionrank = pke.unsupervised.PositionRank()

# 2. load the content of the document.
extractor_positionrank.load_document(input=text,
                                     language='ru',
                                     normalization=None)

# 3. select the noun phrases up to 3 words as keyphrase candidates.
extractor_positionrank.candidate_selection(grammar=grammar,
                                           maximum_word_number=3)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk biaised with the position of the words
#    in the document. In the graph, nodes are words (nouns and
#    adjectives only) that are connected if they occur in a window of
#    10 words.
extractor_positionrank.candidate_weighting(window=3,
                                           pos=pos)


CPU times: user 699 ms, sys: 12 ms, total: 711 ms
Wall time: 711 ms


In [18]:
# 5. get the 100-highest scored candidates as keyphrases
keyphrases_positionrank = extractor_positionrank.get_n_best(n=100)
keyphrases_positionrank

[('- студентов электив', 0.04949750098212494),
 ('электив - чудо', 0.04769862772363934),
 ('электив - пытка', 0.04768441378542007),
 ('- электив', 0.04755494839706157),
 ('электив -', 0.04755494839706157),
 ('данный электив -', 0.04755494839706157),
 ('общем - электив', 0.04755494839706157),
 ('немецкий - электив', 0.04755494839706157),
 ('преподаватель - практик', 0.04360109306796812),
 ('преподаватель - душка', 0.04336415772984271),
 ('преподаватель - психолог', 0.0432091876634155),
 ('практики -', 0.043209146429337986),
 ('преподаватель - итальянец', 0.04317489612467537),
 ('преподаватель - крутая', 0.04317260907385483),
 ('преподаватель - восхитительная', 0.0431721509611779),
 ('преподаватель - замечательная', 0.043170244077198903),
 ('преподаватель -', 0.043087939143672656),
 ('последняя - преподаватель', 0.043087939143672656),
 ('сам преподаватель -', 0.043087939143672656),
 ('решение -', 0.040355601226103086),
 ('хорошие баллы -', 0.03770409596735608),
 ('баллы -', 0.03770409596

# MultipartiteRank

In [19]:
%%time

# 1. create a MultipartiteRank extractor.
extractor_multipartiterank = pke.unsupervised.MultipartiteRank()

stoplist = list(string.punctuation)
stoplist += pke.lang.stopwords.get('ru')

# 2. load the content of the document.
extractor_multipartiterank.load_document(input=text,
                                         language="ru",
                                         stoplist=stoplist)

# 3. select the longest sequences of nouns and adjectives, that do
#    not contain punctuation marks or stopwords as candidates.
pos = {'NOUN', 'PROPN'}
extractor_multipartiterank.candidate_selection(pos=pos)

# 4. build the Multipartite graph and rank candidates using random
#    walk, alpha controls the weight adjustment mechanism, see
#    TopicRank for threshold/method parameters.
extractor_multipartiterank.candidate_weighting(alpha=1.5,
                                               threshold=0.25,
                                               method='average')


CPU times: user 3min 34s, sys: 3.38 s, total: 3min 37s
Wall time: 3min 37s


In [20]:
# 5. get the 100-highest scored candidates as keyphrases
keyphrases_multipartiterank = extractor_multipartiterank.get_n_best(n=100)
keyphrases_multipartiterank

[('электив', 0.04396019369092936),
 ('преподаватель', 0.02900658581829021),
 ('пары', 0.026129806435289612),
 ('задания', 0.018559400673829638),
 ('лекции', 0.013917285369968684),
 ('баллы', 0.012479448361265098),
 ('работы', 0.011411086970617202),
 ('тему', 0.011156552631613371),
 ('элективом', 0.010586393281065514),
 ('занятия', 0.00999777070771723),
 ('практики', 0.009351394465449001),
 ('курса', 0.008561035156433838),
 ('студентам', 0.008334682914741041),
 ('информации', 0.00824556707704251),
 ('презентации', 0.007998489818676057),
 ('зачёт', 0.007699140665131875),
 ('тест', 0.007364762826917305),
 ('вопросы', 0.0070892181209369284),
 ('материал', 0.00659515758094268),
 ('знаний', 0.006450256529949731),
 ('целом', 0.005521373548541383),
 ('предмета', 0.005333789527218861),
 ('группы', 0.004687055591933629),
 ('проект', 0.004618187283318999),
 ('языке', 0.004604878246350171),
 ('время', 0.004074876762442811),
 ('фильмы', 0.00391144952933215),
 ('преподавательница', 0.003843333414933